# Grafische Oberfläche

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
import math

In [ ]:
import logging

# create logger with ''
logger = logging.getLogger('GUI')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler('log.txt')
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)

## Konstanten

(TODO: Konstanten erklären und Schema erstellen)

In [ ]:
# Board constants
BOARD_SIZE = 500
CANVAS_PADDING = 30
ROW_WIDTH = 60
PLAYER_PIECE_RADIUS = 12
DEFAULT_PIECE_RADIUS = 5

DEFAULT_STONE_LINE_WIDTH = 1
SELECTED_STONE_LINE_WIDTH = 3
POUNDED_STONE_LINE_WIDTH = 5

STASH_STONES_SPACING = 5

STASH_HEIGHT = ((PLAYER_PIECE_RADIUS * 2 ) * 2) + (STASH_STONES_SPACING) + (CANVAS_PADDING * 2)
STASH_WIDTH = ((PLAYER_PIECE_RADIUS * 2 ) * 9) + (STASH_STONES_SPACING * 8) + (CANVAS_PADDING * 2)

CANVAS_HEIGHT = BOARD_SIZE
CANVAS_WIDTH = 1500

STASH_STARTING_POINT_X = BOARD_SIZE
STASH_STARTING_POINT_Y = CANVAS_HEIGHT - STASH_HEIGHT

# text
TEXT_X = BOARD_SIZE + CANVAS_PADDING
TEXT_Y = CANVAS_PADDING
TEXT_MAX_WIDTH = CANVAS_WIDTH - BOARD_SIZE - 2 * CANVAS_PADDING
TEXT_FONT = '14px sans-serif'
TEXT_VERTICAL_PADDING = 40


# players
NO_PLAYER = ' '
PLAYER_1 = 'w'
PLAYER_2 = 'b'


# color constants
CANVAS_BACKGROUND_COLOR = '#ffffff'
BOARD_FOREGROUND_COLOR = '#191919'
BOARD_BACKGROUND_COLOR = '#ffffcb'
STASH_BACKGROUND_COLOR = '#dedede'
PLAYER_1_FILL_COLOR = '#E8E8E8'
PLAYER_1_STROKE_COLOR = '#191919'
PLAYER_2_FILL_COLOR = '#5c5c5c'
PLAYER_2_STROKE_COLOR = '#191919'
TEXT_COLOR = '#000000'
LAST_MOVED_COLOR = '#61aced'

# enabled shadow constants
SHADOW_COLOR_ENABLED = '#000000'
SHADOW_OFFSET_X_ENABLED = 2
SHADOW_OFFSET_Y_ENABLED = 2
SHADOW_BLUR_ENABLED = 2

# disabled shadow constants
SHADOW_COLOR_DISABLED = 'rgba(0, 0, 0, 0)'
SHADOW_OFFSET_X_DISABLED = 0
SHADOW_OFFSET_Y_DISABLED = 0
SHADOW_BLUR_DISABLED = 0


### Berechnete Werte

Die Koordinaten für die Knoten lassen sich aus den obrigen Konstanten berechnen. Da das Mühlespielbrett horizontal und vertikal identisch ist, werden für die Koordinaten auf der x- und y-Achse die gleichen Werte benötigt, die mit $av$ (für _available values_) bezeichnet werden. Es werden insgesamt sieben Werte $av_0$ bis $av_6$ benötigt, die in der folgenden Abbildung dargestellt werden.

![](images/nmm-av.png)

Die Werte lassen sich wie folgt berechnen.

$$ av_0 =  CANVAS\_PADDING $$
$$ av_1 =  CANVAS\_PADDING + ROW\_WIDTH $$
$$ av_2 =  CANVAS\_PADDING + 2 \cdot ROW\_WIDTH $$
$$ av_3 =  \frac{BOARD\_SIZE}{2}$$
$$ av_4 =  BOARD\_SIZE - (CANVAS\_PADDING + 2 \cdot ROW\_WIDTH) $$
$$ av_5 =  BOARD\_SIZE - (CANVAS\_PADDING + ROW\_WIDTH)$$
$$ av_6 =  BOARD\_SIZE - CANVAS\_PADDING $$

In [ ]:
av = (
    math.floor(CANVAS_PADDING),
    math.floor(CANVAS_PADDING + ROW_WIDTH),
    math.floor(CANVAS_PADDING + 2 * ROW_WIDTH),
    math.floor(BOARD_SIZE / 2),
    math.floor(BOARD_SIZE - (CANVAS_PADDING + 2 * ROW_WIDTH)),
    math.floor(BOARD_SIZE - (CANVAS_PADDING + ROW_WIDTH)),
    math.floor(BOARD_SIZE - CANVAS_PADDING)
)


### Koordinaten

Die Koordinaten der Knoten sind in dem zweidimensionalen Tupel `coords` definiert. Zuerst wird der Ring definiert, von außen nach innen. Danach die Position im Ring, beginnend von oben links und dann im Uhrzeigersinn. In der Abbildung sind die Knoten mit den Koordinaten dargestellt. Die Werte von den Koordinaten sind die x- und y-Werte auf der Zeichenfläche, definiert in `av`.

![](images/nmm-coords.png)

In [ ]:
coords = (
    (
        (av[0], av[0]),
        (av[3], av[0]),
        (av[6], av[0]),
        (av[6], av[3]),
        (av[6], av[6]),
        (av[3], av[6]),
        (av[0], av[6]),
        (av[0], av[3])
    ),
    (
        (av[1], av[1]),
        (av[3], av[1]),
        (av[5], av[1]),
        (av[5], av[3]),
        (av[5], av[5]),
        (av[3], av[5]),
        (av[1], av[5]),
        (av[1], av[3])
    ),
    (
        (av[2], av[2]),
        (av[3], av[2]),
        (av[4], av[2]),
        (av[4], av[3]),
        (av[4], av[4]),
        (av[3], av[4]),
        (av[2], av[4]),
        (av[2], av[3])
    )
) 

## Funktionen zum Zeichnen

Die grafische Oberfläche (englisch _graphical user interface_, GUI) wird mit dem Python-Modul ipycanvas aufgebaut. Dieses Modul ermöglicht die Verwendung einer interaktiven Zeichenfläche zum Zeichnen von 2D-Objekten in IPython. Es bringt eine Reihe von Funktionen mit, um einfache Formen zeichnen zu können. Gezeichnet wird auf einem 2D-Canvas-Objekt mit den Startkoordinaten `(0,0)` oben links.

In [ ]:
import ipycanvas

Die Funktion `toggleShadow` schaltet den Schatten auf einem gegebenen Canvas ein und aus. Sie hat folgende Eingabeparameter:

- `c` ist eine Referenz auf ein Canvas-Objekt.
- `enable` ist ein boolischer Wert, der angibt, ob Schatten auf dem Canvas `c` ein oder ausgeschaltet werden soll.

Wird der Schatten eingeschaltet, werden die Schatteneigenschaften des Canvas mit den oben definierten Konstanten gesetzt. Andernfalls werden die Standardwerte von _ipycanvas_ gesetzt, was bedeutet, der Schatten wird deaktiviert. 

In [ ]:
def toggleShadow(c, enable):
    c.shadow_color    = SHADOW_COLOR_ENABLED    if enable else SHADOW_COLOR_DISABLED
    c.shadow_offset_x = SHADOW_OFFSET_X_ENABLED if enable else SHADOW_OFFSET_X_DISABLED
    c.shadow_offset_y = SHADOW_OFFSET_Y_ENABLED if enable else SHADOW_OFFSET_Y_DISABLED
    c.shadow_blur     = SHADOW_BLUR_ENABLED     if enable else SHADOW_BLUR_DISABLED

Die Funktion `drawCircle` dient zum Zeichnen eines Kreises auf einem Zeichenfeld. Die Funktion hat vier Argumente und drei optionale Parameter:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem der Kreis gezeichnet werden soll.
- `coords` ist die Koordinate des Mittelpunktes des Kreies.
- `radius` ist der Radius des Kreises.
- `color` gibt die Farbe des Kreises an.
- `strokeColor` ist ein optionaler Parameter, der die Farbe der Umrandung angibt. Der Standardwert ist `None`. In dem Fall wird der Kreis nicht umrandet.
- `lineWidth` ist ein optionaler Parameter, der die Liniendicke angibt. Der Standardwert ist in der Kontante `DEFAULT_STONE_LINE_WIDTH` definiert.
- `useShadow` ist ein optionaler, boolischer Wert. Wenn er gesetzt ist, wird ein Schatten von dem Kreis gemalt. Standardmäßig ist der Wert `False`.

In [ ]:
def drawCircle(c, coords, radius, color, strokeColor = None, lineWidth = DEFAULT_STONE_LINE_WIDTH, useShadow = False):
    if useShadow:
        toggleShadow(c, True)
    c.fill_style = color
    c.fill_arc(coords[0], coords[1], radius, 0, 2 * math.pi)
    if useShadow:
        toggleShadow(c, False)
    if strokeColor is not None:
        c.line_width = lineWidth
        c.stroke_style = strokeColor
        c.stroke_arc(coords[0], coords[1], radius, 0, 2 * math.pi)

Die Funktion `drawStone` dient zum Zeichnen eines Steines auf einem Zeichenfeld mit Hilfe der Funktion `drawCircle`. Ein Spielstein besteht aus zwei Kreisen und ein leerer Knoten (also wo sich kein Spieler befinden) aus einem Kreis.

Die Funktion hat drei Argumente und zwei optionale Parameter:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem der Stein gezeichnet werden soll;
- `coords` ist die Koordinate des Mittelpunktes des Steines;
- `player` gibt  den Spieler an;
- `selected` ist ein optionaler, boolischer Wert, der angibt, ob ein Spielerstein ausgewählt ist oder nicht. Der Standardwert ist `False`;
- `lastMoved` ist ein optionaler, boolischer Wert, der angibt ob der zu zeichnende Spielerstein zuletzt bewegt worden ist. Der Standardwert ist `False`.

In [ ]:
def drawStone(c, coords, player, selected = False, lastMoved = False):
    if player == NO_PLAYER:
        drawCircle(c, coords, DEFAULT_PIECE_RADIUS, BOARD_FOREGROUND_COLOR)
    else:
        color       = PLAYER_1_FILL_COLOR   if player == PLAYER_1 else PLAYER_2_FILL_COLOR
        strokeColor = PLAYER_1_STROKE_COLOR if player == PLAYER_1 else PLAYER_2_STROKE_COLOR
        
        lineWidth = SELECTED_STONE_LINE_WIDTH if selected else DEFAULT_STONE_LINE_WIDTH
        drawCircle(c, coords, PLAYER_PIECE_RADIUS                , LAST_MOVED_COLOR if lastMoved else color, strokeColor = strokeColor, lineWidth = lineWidth, useShadow = True)
        drawCircle(c, coords, math.floor(PLAYER_PIECE_RADIUS / 2), color, strokeColor = strokeColor, lineWidth = lineWidth)

Die Funktion `drawPoundedStone` dient zum Zeichnen eines geschlagenden Steines auf einem Zeichenfeld. Ein geschlagender Stein wird durch ein Kreuz in der GUI dargestellt.

Die Funktion hat drei Argumente:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem der geschlagende Stein gezeichnet werden soll;
- `coords` ist die Koordinate des Mittelpunktes des geschlagenden Steines;
- `player` gibt  den Spieler des Steines an.

In [ ]:
def drawPoundedStone(c, coords, player):
    x, y = coords
    offset = int(math.sin(0.25*math.pi) * PLAYER_PIECE_RADIUS)
    c.line_width = POUNDED_STONE_LINE_WIDTH
    c.stroke_style = PLAYER_1_FILL_COLOR   if player == PLAYER_1 else PLAYER_2_FILL_COLOR
    with ipycanvas.hold_canvas(c):
        c.stroke_line(x - offset, y - offset, x + offset, y + offset)
        c.stroke_line(x - offset, y + offset, x + offset, y - offset)
    

Die Funktion `drawText` zeichnet einen Text auf einer Zeichenfläche. Die Funktion hat zwei Argumente und zwei optionale Parameter:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem der Text gezeichnet werden soll.
- `msg` ist die Nachricht, die auf dem Canvas geschrieben werden soll.
- `hint` ist ein optionaler String, der ein Hinweis oder eine Warnung ist. Standardmäßig ist die Variable auf `None` gesetzt.
- `scoreMsg` ist ein optionaler String, der den ermittelten Score des gespielten Zuges vom Computer angibt. Standardmäßig ist die Variable auf `None` gesetzt.

Bei jedem Funktionsaufruf wird am Anfang der Inhalt der Zeichenfläche gelöscht, sodass sich immer nur eine Version der Texte auf der Zeichenfläche befindet.  

In [ ]:
def drawText(c, msg, hint = None, scoreMsg = None):
    with ipycanvas.hold_canvas(c):
        c.clear()
        y = TEXT_Y
        c.fill_text(msg, TEXT_X, y, max_width = TEXT_MAX_WIDTH)
        y += TEXT_VERTICAL_PADDING
        if hint is not None:
            c.fill_text('hint: ' + hint, TEXT_X, y, max_width = TEXT_MAX_WIDTH)
            y += TEXT_VERTICAL_PADDING
        if scoreMsg is not None:
            c.fill_text('score: ' + scoreMsg, TEXT_X, y, max_width = TEXT_MAX_WIDTH)

Die Funktion `constructLine` dient zum Konstruieren einer Linie auf einem Zeichenfeld. Die Funktion hat drei Eingabeparameter:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem die Linie gezeichnet werden soll.
- `start` ist die Koordinate des Startpunktes der Linie.
- `end` ist die Koordinate des Endpunktes der Linie.

Die Funktion aktualisiert einen Pfad auf dem Zeichenfeld, aber sie zeichnet noch nicht den aktualisierten Pfad.

In [ ]:
def constructLine(c, start, end):
    c.move_to(start[0], start[1])
    c.line_to(end[0], end[1])

Die Funktion `constructSquare` konstruiert ein Quadrat auf einer Zeichenfläche und lässt es mit Hilfe der Funktion `constructLine` zeichnen. Die Funktion hat zwei Eingabeargumente:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem das Quadrat gezeichnet werden soll.
- `ring` ist ein Acht-Tupel, das die Koordinaten eines Ringes enthält.

In [ ]:
# Function to construct a square on a canvas c.
def constructSquare(c, ring):
    for i in range(4):
        start = i * 2
        end = (i * 2 + 2) if (i * 2 + 2 <= 6) else 0 
        constructLine(c, ring[start], ring[end])

Die Funktion `constructCrossLines` konstruiert die Querlinien des Mühlespiels auf einer Zeichenfläche und lässt es mit Hilfe der Funktion `constructLine` zeichnen. Die Funktion hat zwei Eingabeargumente:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem die Querlinien gezeichnet werden sollen.
- `coords` ist ein zweidimensionales Tupel, welches alle Koordinaten des Spielbrettes enthält (vgl. das Kapitel _Koordinaten_ in der GUI).

In [ ]:
def constructCrossLines(c, coords):
    for i in range(4):
        k = i * 2 + 1
        constructLine(c, coords[0][k], coords[2][k])

Die Funktion `setupCanvas` erstellt das Canvas-Objekt und zeichnet den Hintergrund des Spielfeldes. Die Funktion hat keine Eingabeparameter und gibt eine Referenz auf das erstellte Canvas-Objekt zurück.

Die Zeichenfläche besteht aus einem MultiCanvas-Objekt mit drei Ebenen:

- Der Hintergrund, der das Spielbrett mit den Linien darstellt;
- Auf der zweiten Ebene wird der Text für das Spiel geschrieben;
- Die Spielsteine werden auf der obersten Ebene gezeichnet. 

In [ ]:
def setupCanvas():
    canvas = ipycanvas.MultiCanvas(3, width = CANVAS_WIDTH, height = CANVAS_HEIGHT)
    with ipycanvas.hold_canvas(canvas[0]):
        
        canvas[0].fill_style = CANVAS_BACKGROUND_COLOR
        canvas[0].fill_rect(0, 0, CANVAS_WIDTH, CANVAS_HEIGHT)
        
        canvas[0].fill_style = BOARD_BACKGROUND_COLOR
        canvas[0].fill_rect(0, 0, BOARD_SIZE, BOARD_SIZE)

        canvas[0].fill_style = STASH_BACKGROUND_COLOR
        canvas[0].fill_rect(STASH_STARTING_POINT_X, STASH_STARTING_POINT_Y, STASH_WIDTH, STASH_HEIGHT)


        canvas[0].stroke_style = BOARD_FOREGROUND_COLOR
        canvas[0].begin_path()

        for i in range(3):
            constructSquare(canvas[0], coords[i])

        constructCrossLines(canvas[0], coords)
        canvas[0].stroke()
    
    canvas[1].font = TEXT_FONT
    canvas[1].fill_style = TEXT_COLOR
    
    return canvas

Die Funktion `updateGui` dient zum Aktualisieren der Zeichenfläche für einen gegebenen Spielzustand. Die Funktion hat zwei Argumente und drei optionale Parameter:

- `c` ist eine Referenz auf ein Canvas-Objekt;
- `state` ist der Spielzustand, der in der GUI angezeigt werden soll;
- `selectedStone` ist eine Koordinate von einem selektierten Stein. Der Standardwert ist `None`. Ist in der Phase 2 oder 3 ein Stein auswählt, kann er mit diesem optionalen Parameter auf der Zeichenfläche hervorgehoben werden;
- `movedStone` ist die Koordinate des zuletzt bewegten Steins, um diesen in der GUI entsprechend zu markieren. Der Standardwert ist `None`, was bedeutet, das kein Stein bewegt worden ist;
- `poundedStones` ist eine Menge von den geschlagenden Steinen des Gegenspielers. Diese werden in der GUI als ein Kreuz dargestellt. Standardmäßig ist `poundedStones` eine leere Menge, was bedeutet, das kein Stein geschlagen worden ist. 

In [ ]:
def updateGui(c, state, selectedStone = None, movedStone = None, poundedStones = set()):
    with ipycanvas.hold_canvas(c):
        c.clear()
        ((stashP1, stashP2), squares) = state

        # update pieces on the board
        for i in range(len(squares)):
            for j in range(len(squares[i])):
                drawStone(c, coords[i][j], squares[i][j], selected = selectedStone == (i, j), lastMoved = movedStone == (i, j))

        for (player, (i, j)) in poundedStones:
            drawPoundedStone(c, coords[i][j], player)
            
        # update pieces on the stash

        # player 1

        x = STASH_STARTING_POINT_X + PLAYER_PIECE_RADIUS
        y = STASH_STARTING_POINT_Y + CANVAS_PADDING + PLAYER_PIECE_RADIUS

        for i in range(stashP1):
            x += 2 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING
            drawStone(c, (x, y), PLAYER_1)

        # player 2

        x = STASH_STARTING_POINT_X + PLAYER_PIECE_RADIUS
        y = STASH_STARTING_POINT_Y + CANVAS_PADDING + 3 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING

        for i in range(stashP2):
            x += 2 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING
            drawStone(c, (x, y), PLAYER_2)
    

## Hilfsfunktionen für das Spielen in der GUI

In diesem Kapitel werden Funktionen deklariert, die Hilfsfunktionen für die GUI darstellen, aber unabhängig von dem eigentlichen Spielzustand sind.

Zusätzlich werden die Jupyter-Notebooks von dem Minimax- und Alpha-Beta-Pruning-Algorithmus benötigt und hier ausführt.

In [ ]:
%run ./nmm-minimax.ipynb
%run ./nmm-alpha-beta-pruning.ipynb

Die Funktion `getClickedStone` dient zum Ermitteln, ob auf der Zeichenfläche eine Ecke angeklickt worden ist, auf dem ein Stein stehen kann. Die Funktion hat zwei Argumente:

- `x` für den Wert auf der horizontalen Achse;
- `y` für den Wert auf der vertikalen Achse.

Es müssen nicht die genauen Werte für x und y angeklickt werden, sondern es gibt einen Puffer in Höhe des Radius von einem Spielerstein. Falls eine Position für einen Stein angeklickt worden ist, für die jeweilige Koordinate aus dem `coords`-Tupel zurückgegeben. Falls keine Position gefunden worden ist, wird `None` zurückgegeben.

In [ ]:
def getClickedStone(x, y):
    for value in av:
        if value - PLAYER_PIECE_RADIUS <= x <= value + PLAYER_PIECE_RADIUS:
            logger.info(f'value for x = {x} found: {value}')
            x = value
        if value - PLAYER_PIECE_RADIUS <= y <= value + PLAYER_PIECE_RADIUS:
            logger.info(f'value for y = {y} found: {value}')
            y = value

    for i in range(len(coords)):
        for j in range(len(coords[i])):
            if coords[i][j] == (x, y):
                logger.info(f'stone clicked: {(x, y)}')
                return (i, j)
    return None

Die Funktion `getChangedStones` ermittelt den zuletzt bewegten Stein eines Spielers und alle geschlagenden Steine des Gegenspielers zwischen zwei Zuständen. Die Funktion hat drei Argumente:

- `oldState` ist der Ausgangszustand;
- `newState` ist der neue Zustand;
- `player` ist der Spieler, den den Zug gespielt hat.

Die Funktion gibt ein Zwei-Tupel der Form `<movedStone, poundedStones>` mit
- `movedStone` ist die Koordinate des bewegten Steins des Spielers;
- `poundedStones` ist eine Menge von Zwei-Tupeln der Form `<op, coord>` mit 
    - `op` ist der Gegenspieler;
    - `coord` ist die Koordinate des geschlagenden Steines;

zurück.

In [ ]:
def getChangedStones(oldState, newState, player):
    (_, oldBoard) = oldState
    (_, newBoard) = newState
    op = opponent(player)
    movedStone = None;
    poundedStones = set()
    
    for i in range(len(oldBoard)):
        for j in range(len(oldBoard[i])):
            if oldBoard[i][j] != newBoard[i][j]:
                if newBoard[i][j] == player:
                    movedStone = (i, j)
                if oldBoard[i][j] == op:
                    poundedStones |= { (op, (i, j)) }
    return (movedStone, poundedStones)

## Klasse GameState

Die Klasse `GameState` dient zum Verwalten von einem Mühle-Spiel. Sie definiert Funktionen, um das Spiel in der GUI spielen zu können.

Der Konstruktur der Klasse hat elf Eingabeparameter, die alle optional sind:
- `state` ist der Startzustand für das Spiel. Standardmäßig wird das Spiel mit `s0` gestartet, welches ein leeres Spielfeld darstellt;
- `player` definiert den Spieler, der den ersten Zug spielt. Standardmäßig wird das Spiel mit dem weißen Spieler (`w`) gestartet;
- `algorithm1` definiert, ob und welcher Algorithmus für den weißen Spieler spielt. Standardmäßig wird der Spieler von einem Menschen gespielt. Für Minimax ist der String `'MM'` zu nehmen und für α-β-Pruning `'AB'`;
- `algorithm1Limit` definiert das Limit der Suchtiefe, mit welchem der Algorithmus für den weißen Spieler spielt. Standardmäßig ist das Limit auf `None` gesetzt, sodass das Limit in dem Algorithmus selber bestimmt wird. Das Limit hat nur bei dem Algorithmus Minimax eine Verwendung, da α-β-Pruning eine iterative Tiefensuche verwendet;
- `algorithm2` definiert, ob und welcher Algorithmus für den schwarzen Spieler spielt. Standardmäßig wird der Spieler von einem Menschen gespielt. Für Minimax ist der String `'MM'` zu nehmen und für α-β-Pruning `'AB'`;
- `algorithm2Limit` definiert das Limit der Suchtiefe, mit welchem der Algorithmus für den schwarzen Spieler spielt. Standardmäßig ist das Limit auf `None` gesetzt, sodass das Limit in dem Algorithmus selber bestimmt wird. Das Limit hat nur bei dem Algorithmus Minimax eine Verwendung, da α-β-Pruning eine iterative Tiefensuche verwendet;
- `timeout` definiert einen Timeout in Sekunden, der nach einem Computer-Zug gesetzt wird. Dies dient dazu, die Übersichtlichkeit zu erhöhen, wenn Computer gegen Computer spielt. Standardmäßig gibt es keinen Timeout (`None`);
- `stepwise` ist ein boolischer Wert und gibt an, ob das Spiel bei Computer gegen Computer im Einzelschrittmodus gespielt wird. Das bedeutet, nach jedem Computerzug muss der nächste Computerzug manuell begonnen werden. Standardmäßig ist der Einzelschrittmodus deaktiviert (`False`);
- `limitMovesWithoutMill` ist eine Ganzzahl, die angibt, wie viele Züge ohne geschlagende Mühle gespielt werden können. Ist das Limit überschritten, endet das Spiel in einem Unentschieden. Der Standardwert ist `30`. Um dieses Limit auszuschalten, muss der Wert auf `None` gesetzt werden;
- `limitStatesCounter` ist eine Ganzzahl, die angibt, wie oft ein gleicher Zustand gespielt werden kann. Ist das Limit überschritten, endet das Spiel in einem Unentschieden. Der Standardwert ist `5`. Um dieses Limit auszuschalten, muss der Wert auf `None` gesetzt werden;

In [ ]:
from collections import defaultdict

class GameState:
    def __init__(self, state = s0, player = PLAYER_1, algorithm1 = None, algorithm1Limit = None, algorithm2 = None, algorithm2Limit = None, timeout = None, stepwise = False, limitMovesWithoutMill = 30, limitStatesCounter = 5):
        self.state = state
        self.player = player
        self.canvas = setupCanvas()
        self.winner = None
        self.score = None
        self.resetStateVariables()
        
        self.algorithm1 = algorithm1.upper() if algorithm1 else None
        self.algorithm1Limit = algorithm1Limit
        self.algorithm2 = algorithm2.upper() if algorithm2 else None
        self.algorithm2Limit = algorithm2Limit
        self.timeout = timeout
        self.stepwise = stepwise
        self.limitMovesWithoutMill = limitMovesWithoutMill
        self.movesWithoutMill = 0
        self.limitStatesCounter = limitStatesCounter
        self.statesCounter = defaultdict(int)
        
        self.pause = (self.player == PLAYER_1 and self.algorithm1) or (self.player == PLAYER_2 and self.algorithm2)
        if self.pause:
            self.hint = 'Please click to start the game.'
        
        self.canvas[2].on_mouse_up(self.handleGame)
        
        updateGui(self.canvas[2], self.state)
        self.updateText()
        logger.info('game state initalized')
        

Die Funktion `resetStateVariables` dient zum Zurücksetzen der temporären Hilfsvariablen der Klasse `GameState`. Diese Funktion hat weder Ein- noch Ausgabe.

In [ ]:
def resetStateVariables(self):
    self.stateTemp = None
    self.millsToPound = 0
    self.selectedStone = None
    self.hint = None
    
GameState.resetStateVariables = resetStateVariables
del resetStateVariables

Die Funktion `handleGame` steuert den Ablauf des Spiels. Die Funktion wird bei jedem Mausklick auf das Canvas-Objekt von dem Event `on_mouse_up` aufgerufen. Die Funktion hat zwei Argumente:

- `x` ist relative Wert der Maus zu dem Canvas-Objekt auf der horizontalen Achse;
- `y` ist relative Wert der Maus zu dem Canvas-Objekt auf der vertikalen Achse.

In [ ]:
def handleGame(self, x, y):
    if self.winner is not None:
        logger.warning('Game has ended!')
        return
    
    if self.pause:
        self.pause = False
        self.hint = None
        
        self.updateText()
        self.checkForComputerStep()
    
        return
    
    self.score = None
    logger.info(f'({x}, {y})')
    phase = playerPhase(self.state, self.player)
    logger.info(f'player phase: {phase}')
    
    stone = getClickedStone(x, y)

    if stone is None:
        logger.warning('No stone was clicked!')
        if self.selectedStone is not None and self.millsToPound <= 0:
            self.cancelStep()
    elif self.millsToPound > 0:
        self.poundMillInGui(stone)
    elif self.selectedStone is not None:
        self.moveStone(stone)
    elif phase == 1:
        self.placeStone(stone)
    elif phase == 2 or phase == 3:
        self.selectStone(stone)

    self.updateText()

    self.checkForComputerStep()

GameState.handleGame = handleGame
del handleGame

Die Funktion `togglePlayer` tauscht den Spieler, der den nächsten Zug spielt.

In [ ]:
def togglePlayer(self):
    self.player = opponent(self.player)
        
GameState.togglePlayer = togglePlayer
del togglePlayer

Die Funktion `playNewState` spielt einen vollständigen Zug in der GUI. Dafür hat sie ein Argument:

- `newState` ist der neue Zustand, der gespielt werden soll.

Die Funktion aktualisiert alle benötigten Hilfsvariablen in der Klasse `GameState` und ruft diverse Funktionen auf, um das Spiel für den neuen Zug vorzubereiten.

Sind die Regeln für ein Untenschieden `limitStatesCounter` und `limitMovesWithoutMill` aktiviert (also nicht `None`), werden die entsprechenden Zähler, um die Regeln zu kontrollieren, aktualisiert. Die Regeln werden jedoch nur aktualisiert, wenn sich die Spieler nicht mehr in der Setzphase befinden.

In [ ]:
def playNewState(self, newState): 
    movedStone, poundedStones = getChangedStones(self.state, newState, self.player)
    phase = playerPhase(self.state, self.player)
    if phase != 1:
        if self.limitStatesCounter is not None:
            self.statesCounter[newState] += 1
            logger.info(f'the state {newState} was played {self.statesCounter[newState]} times.')

        if self.limitMovesWithoutMill is not None:
            if (len(poundedStones) == 0):
                self.movesWithoutMill += 1
            else:
                self.movesWithoutMill = 0
            logger.info(f'moves without mill: {self.movesWithoutMill}')
        
    self.state = newState
    self.resetStateVariables()
    self.togglePlayer()
    self.checkIfFinished()
    
    if phase != 1:
        if (self.limitStatesCounter is not None) and (self.statesCounter[self.state] + 1 >= self.limitStatesCounter):
            self.hint = f'The state has already been played {self.statesCounter[newState]} times. If it is played once more, the game is a tie!'

        if (self.limitMovesWithoutMill is not None) and (self.movesWithoutMill + 5 >= self.limitMovesWithoutMill):
            self.hint = f'no mill was pound in the last {self.movesWithoutMill} moves. In {self.limitMovesWithoutMill - self.movesWithoutMill} moves the game is a tie!'
            
    updateGui(self.canvas[2], self.state, movedStone = movedStone, poundedStones = poundedStones)
    self.updateText()

GameState.playNewState = playNewState
del playNewState

Die Funktion `checkForComputerStep` überprüft, ob der nächste Zug von einem Algorithmus gespielt werden soll. Falls dies der Fall ist, wird der jeweilige Algorithmus ausgeführt und der Spielzustand aktualisiert.

In [ ]:
from time import sleep

def checkForComputerStep(self):
    if self.winner is not None:
        logger.warning('Game has ended!')
        return
    
    if self.pause:
        logger.warning('Pause!')
        self.hint = 'The game has paused. Please click to continue!'
        self.updateText()
        return
    
    if (self.player == PLAYER_1 and self.algorithm1) or (self.player == PLAYER_2 and self.algorithm2):
        logger.info(f'computer calculating for player {self.player} with algorithm {self.algorithm1 if self.player == PLAYER_1 else self.algorithm2}')
        
        if (self.player == PLAYER_1 and self.algorithm1 == 'MM') or (self.player == PLAYER_2 and self.algorithm2 == 'MM'):
            bestValue, bestMove = bestMove_MM(self.state, self.player, limit = self.algorithm1Limit if self.player == PLAYER_1 else self.algorithm2Limit)
        elif (self.player == PLAYER_1 and self.algorithm1 == 'AB') or (self.player == PLAYER_2 and self.algorithm2 == 'AB'):
            bestValue, bestMove = bestMove_AB(self.state, self.player)
        else:
            logger.error(f'Invalid algorithm: {self.algorithm1} or {self.algorithm2}!')
            return
        self.score = bestValue
        logger.info(f'best state calculated with Score {bestValue}:\n{bestMove}')
        
        self.playNewState(bestMove)
        
        if self.timeout:
            self.hint = f'Timeout ({self.timeout} seconds). Please wait!'
            self.updateText()
            sleep(self.timeout)
        
        self.pause = self.stepwise
        self.checkForComputerStep()
    
GameState.checkForComputerStep = checkForComputerStep
del checkForComputerStep

Die Funktion `placeStone` dient zum Platzieren eines Spielersteins in der Spielphase 1. Die Funktion hat ein Argument:

- `coord` ist die Koordinate aus dem Tupel `coords` an dem der Stein des Spielers gesetzt werden soll.

In [ ]:
def placeStone(self, coord):
    if getPlayerAt(self.state[1], coord) != NO_PLAYER:
        logger.warning(f'{coord} is not free')
        self.hint = f'The slot at {coord} is not free!'
        return

    newState = (removeFromStash(self.state[0], self.player), place(self.state[1], coord, self.player))

    if self.validateNewState(newState):
        logging.info('stone placed')
    else:
        logger.info('NewState not in allAvailableStates, checking for new Mills ...')
        self.checkForNewMills(newState)

        
GameState.placeStone = placeStone
del placeStone

Die Funktion `selectStone` dient zum Selektieren des Steines, der in der Phase 2 verschoben bzw. in Phase 3 springen soll. Die Funktion hat ein Argument:

- `stone` ist die Koordinate des Steines, der bewegt werden soll.

Die Funktion erzeugt bei erfolgreicher Validierung einen Hilfszustand in der GUI mit dem markierten Stein.

In [ ]:
def selectStone(self, stone):
    if getPlayerAt(self.state[1], stone) != self.player:
        logger.warning(f'{stone} is not the own stone')
        self.hint = 'Please select your own stone!'
        return
    self.selectedStone = stone
    self.hint = None
    updateGui(self.canvas[2], self.state, selectedStone = self.selectedStone)

    
GameState.selectStone = selectStone
del selectStone

Die Funktion `moveStone` dient zum Bewegen des selektierten Steins in der Zug- und Endphase. Die Funktion hat ein Argument:

- `coord` ist die Koordinate, wohin der Stein bewegt werden soll.

Der zu bewegende Stein wurde in dem vorherigen Hilfszug in der Funktion `selectStone` ausgewählt und in der Hilfsvariablen `selectedStone` gespeichert.

In [ ]:
def moveStone(self, coord):
    if getPlayerAt(self.state[1], coord) != NO_PLAYER:
        logger.warning(f'{coord} is not free')
        self.hint = f'The slot at {coord} is not free!'
        return
    
    canJump = isAllowedToJump(self.state, self.player)
    logger.info(f'moveStone({coord})')

    if canJump or coord in findNeighboringEmptyCells(self.state[1], self.selectedStone):
        newState = (self.state[0], place(self.state[1], self.selectedStone, NO_PLAYER))
        newState = (newState[0], place(newState[1], coord, self.player))

        if self.validateNewState(newState):
            movement = 'jumped' if canJump else 'moved'
            logger.info(f'Stone successfully {movement}!')
        else:
            logger.info('Round not finished, checking for new mills...')
            self.checkForNewMills(newState)
    else:
        logger.warning(f'{coord} is not a (free) neighbor of {self.selectedStone}!')
        self.hint = f'The slot at {coord} is not a (free) neighbor of {self.selectedStone}!'

GameState.moveStone = moveStone
del moveStone

Die Funktion `checkForNewMills` überprüft, ob ein gegebener Zustand neue Mühlen enthält. Die Funktion hat ein Argument:

- `newState` ist der neue Zustand, der überprüft werden soll.

Falls neue Mühlen gefunden worden sind, wird ein temporärer Zustand erstellt, der den menschlichen Spieler auffordert, einen gegnerischen Stein von dem Spiellbrett zu entfernen. Im Englischen wird dies als _pounding_ bezeichnet.

In [ ]:
def checkForNewMills(self, newState):
    oldMills = findMills(self.state[1], self.player)
    newMills = countNewMills(newState[1], oldMills, self.player)

    if newMills > 0:
        
        logger.info(f'new mills: {newMills}')
        
        self.stateTemp = newState
        self.millsToPound = newMills
        self.hint = None
        movedStone, poundedStones = getChangedStones(self.state, self.stateTemp, self.player)
        updateGui(self.canvas[2], self.stateTemp, movedStone = movedStone, poundedStones = poundedStones)
    

GameState.checkForNewMills = checkForNewMills
del checkForNewMills

Die Funktion `poundMillInGui` entfernt einen gegnerischen Spielerstein und beendet somit einen Mühlenzug. Die Funktion hat dabei ein Argument:

- `stone` ist die Koordinate des gegnerischen Spielersteins, der entfernt werden soll.

Ob ein Spielerstein entfernt werden kann, wird mit der Funktion `validateNewState` validiert.

In der Setzphase kann es vorkommen, dass ein Spieler zwei Mühlen schlagen kann. In diesem Fall kann das nicht von der Funktion `validateNewState` ausführt werden, weil der Zug noch nicht abgeschlossen ist und somit nicht in der Menge `nextStates` auftritt. In diesem Fall muss die Validierung von der Funktion selber durchgeführt werden.

In [ ]:
def poundMillInGui(self, stone):
    if self.millsToPound  <= 0:
        logger.warning('Player has no Mills to pound!')
        return
    if getPlayerAt(self.state[1], stone) != opponent(self.player):
        logger.warning(f'{stone} is not the opponent!')
        self.hint = 'Please select an opponent stone!'
        
        return
    # if the player has only one mill left to pound, it uses the place function and afterwards validates the newState
    if self.millsToPound == 1:
        newState = (self.stateTemp[0], place(self.stateTemp[1], stone, NO_PLAYER))
        if self.validateNewState(newState):
            logger.info('success')
        else:
            logger.warning('Mills could not be pounded! The new state could not be validated by the game logic.')
            self.hint = 'Please do not select an opponent stone that is in a mill!'

        return
    
    # otherwise the gui has to validate the mill manually, as the intermediate step cannot be checked by the game logic
    if stone in getCellsPoundable(self.stateTemp[1], self.player):
        self.stateTemp = (self.stateTemp[0], place(self.stateTemp[1], stone, NO_PLAYER))
        self.millsToPound -= 1
        self.hint = None
        movedStone, poundedStones = getChangedStones(self.state, self.stateTemp, self.player)
        updateGui(self.canvas[2], self.stateTemp, movedStone = movedStone, poundedStones = poundedStones)
    else:
        logger.warning('Mills could not be pounded! The new state could not be validated by the gui.')
        self.hint = 'Please do not select an opponent stone that is in a mill!'

GameState.poundMillInGui = poundMillInGui
del poundMillInGui

Die Funktion `validateNewState` überprüft, ob ein gegebener Zustand in der Menge der `nextStates` vorhanden ist. Die Funktion hat ein Argument:

- `newState` ist der neue Zustand, der validert werden soll.

Falls sich der neue Zustand `newState` in der Menge `nextStates` von dem aktuellen Zustand `state` befindet, ist ein Zug von dem Spieler abgeschlossen. Die Hilfsvariablen werden zurückgesetzt und es wird der Spieler getauscht. 

In [ ]:
def validateNewState(self, newState):
    allAvailableStates = nextStates(self.state, self.player)
    if newState in allAvailableStates:
        logger.info('SUCCESS')
        self.playNewState(newState)
        return True
    else:
        logger.warning('state is not in allAvailableStates!')
        return False

GameState.validateNewState = validateNewState
del validateNewState

Die Funktion `cancelStep` dient zum Deselektieren eines Steines in der Zug- oder Endphase.

In [ ]:
def cancelStep(self):
    logger.warn('step is canceled.')
    
    self.resetStateVariables()
    updateGui(self.canvas[2], self.state)
    
GameState.cancelStep = cancelStep
del cancelStep

Die Funktion `updateText` dient zum Aktualisieren des Textes und des Hinweises auf dem Spielbrett. Die Funktion ermittelt dabei selbständig den aktuellen Zustand des Spieles anhand der Variablen innerhalb der Klasse.

In [ ]:
def updateText(self):
    phase = playerPhase(self.state, self.player)
    if self.winner is None:
        message = f'Player {self.player}: '
        
        if self.player == PLAYER_1 and self.algorithm1:
            message += f'Computer\'s turn with {self.algorithm1}. Please wait.'
        elif self.player == PLAYER_2 and self.algorithm2:
            message += f'Computer\'s turn with {self.algorithm2}. Please wait.'
        elif self.millsToPound == 1:
            message += 'Pound your mill.'
        elif self.millsToPound > 1:
            message += f'You have {self.millsToPound} mills left to pound. Please pound your next mill.'
        elif self.selectedStone is not None:
            movement = 'move' if phase == 2 else 'jump'
            message += f'{movement} your selected stone.'
        elif phase == 1:
            message += 'Place your stone.'
        elif phase == 2 or phase == 3:
            movement = 'move' if phase == 2 else 'jump'
            message += f'select your stone you want to {movement}.'      
    else:
        message = 'The game has ended: '
        self.hint = None
        if self.winner == NO_PLAYER:
            message += 'Tie.'
        else:
            message += f'{self.winner} has won!'
    scoreMsg = f'{opponent(self.player)} played with score {self.score}' if self.score is not None else None
    logger.info(message)
    drawText(self.canvas[1], message, hint = self.hint, scoreMsg = scoreMsg)

GameState.updateText = updateText
del updateText

Die Funktion `checkIfFinished` überprüft, ob ein Spiel beendet worden ist. Dabei benutzt es die Funktionen `finished` und `utility` aus dem Jupyter-Notebookt `nmm-game`.

In [ ]:
def checkIfFinished(self):
    if (self.limitMovesWithoutMill is not None) and (self.movesWithoutMill >= self.limitMovesWithoutMill):
        self.winner = NO_PLAYER
    elif (self.limitStatesCounter is not None) and (self.statesCounter[self.state] >= self.limitStatesCounter):
        self.winner = NO_PLAYER
    elif finished(self.state, self.player):
        status = utility(self.state, self.player)
        if status == 0:
            self.winner = NO_PLAYER
        else:
            self.winner = self.player if status == 1 else opponent(self.player)

GameState.checkIfFinished = checkIfFinished
del checkIfFinished

## Spielen

Um das Spiel in der GUI zu spielen, muss zuerst ein Objekt der Klasse `GameState` initialisiert werden. Die Argumente für die Klasse bestimmen die Spieloptionen.

Anschließend muss noch die dynamische Zeichenfeldreferenz des Objektes ausgegeben werden, um das Spielfeld anzuzeigen.

In [ ]:
gameState = GameState(algorithm1 = 'ab', algorithm2 = 'mm')

In [ ]:
gameState.canvas